# Project data analisys m1

In [1]:
import requests
import json
import pandas as pd
import duckdb
from shapely.geometry import Point
import geopandas as gpd
from modules import geo_calculations
pd.options.display.max_columns = None
import requests

---
### Gardens

In [2]:
# endpoint api Madrid gardens
response = requests.get('https://datos.madrid.es/egob/catalogo/200761-0-parques-jardines.json')
results_api = response.json()

In [3]:
# Selecciono la lista que quiero
list_api_garden = results_api['@graph']

In [4]:
# Transformo a data frame
df_garden = pd.DataFrame(list_api_garden)

In [5]:
# Separo langitud y latitud de la columna location
df_garden_location = df_garden["location"].apply(pd.Series)

In [6]:
# Separa los elementos de address
df_garden_address = df_garden["address"].apply(pd.Series)

In [7]:
# Elimino lo innecesario para hacerlo más legible de @type
# La barra se podría cambiar con un str.replace por una coma
df_garden_type = df_garden["@type"].str.replace('https://datos.madrid.es/egob/kos/entidadesYorganismos/', '')

In [8]:
# En el data frame de garden, me quedo solo con title, para no repetir @type y evitar posibles problemas en el futuro
both_df_garden = pd.concat([df_garden[['title']], df_garden_location, df_garden_address, df_garden_type], axis = 1)

In [9]:
# Elimino columnas o selecciono columnas que quiero: location, type, address and title of gardens, o no
cleaning_df_garden = both_df_garden[['title', '@type', 'locality', 'postal-code', 'street-address', 'latitude', 'longitude']]

---
### Bicimad

In [10]:
# endpoint duck

In [11]:
db_bicimad = duckdb.connect('bicimad.db')

In [12]:
# Data Frame data base statiosn

In [13]:
df_bicimad = db_bicimad.sql('select * from bicimad_stations').df()

In [14]:
# Separo geometry coordinates

In [15]:
df_bicimad[['longitude', 'latitude']] = df_bicimad["geometry.coordinates"].apply(lambda x: pd.Series(str(x).replace('[','').replace(']','').split(",")))

In [16]:
df_bicimad = df_bicimad[['name', 'address', 'latitude', 'longitude',]]

---

## Muestro los dos resultados de data wrangling

In [17]:
df_bicimad

,name,address,latitude,longitude
0,1a - Puerta del Sol A,Puerta del Sol nº 1,40.4172137,-3.7018341
1,1b - Puerta del Sol B,Puerta del Sol nº 1,40.41731271011562,-3.701602938060457
2,2 - Miguel Moya,Calle Miguel Moya nº 1,40.4205886,-3.7058415
3,3 - Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,40.4302937,-3.7069171
4,4 - Malasaña,Calle Manuela Malasaña nº 5,40.4285524,-3.7025875
...,...,...,...,...
259,257 - INEF,Avenida Juan de Herrera frente a la calle Paul...,40.43896,-3.72997
260,258 - Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,40.44375,-3.72699
261,259 - Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,40.44342,-3.72693
262,260 - Facultad Biología,Calle José Antonio Novais frente al nº 12,40.4483322,-3.7272945


In [18]:
cleaning_df_garden

,title,@type,locality,postal-code,street-address,latitude,longitude
0,Jardines Gregorio Ordóñez,ParquesJardines,MADRID,28006,CALLE PRINCIPE DE VERGARA 68,40.433462,-3.678595
1,Jardines San Francisco el Grande,ParquesJardines/ColeccionesBotanicas,MADRID,28005,AVENIDA GRAN VIA DE SAN FRANCISCO 29,40.407143,-3.712167
2,Jardines de Andrés Saborit,ParquesJardines,MADRID,28003,CALLE RAIMUNDO FERNANDEZ VILLAVERDE 32,40.446133,-3.698970
3,Jardines de El Buen Retiro,ParquesJardines/ParquesHistoricos,MADRID,28001,PLAZA INDEPENDENCIA 7,40.419641,-3.687934
4,Jardines de Gloria Fuertes,NaN,MADRID,28016,CALLE MADRE DE DIOS 2,40.461518,-3.675118
...,...,...,...,...,...,...,...
198,Vivero Estufas de El Retiro,ParquesJardines,MADRID,28009,GLORIETA ANGEL CAIDO 4 Parque del Retiro. Entr...,40.409899,-3.682331
199,Vivero de la Casa de Campo,ParquesJardines,MADRID,28011,PASEO AZUL 2,40.419842,-3.728697
200,Vivero municipal de Migas Calientes,NaN,MADRID,28040,AUTOVIA M-30 2200,40.445356,-3.739974
201,Zona verde C/ Fresnedillas,ParquesJardines,MADRID,28035,CALLE FRESNEDILLAS 1 C/ Los Gavilanes y C/ Gab...,40.477264,-3.743003


## Data Analisys

In [19]:
# Incluir de alguna forma la comparacion con un loop o comprehension

In [20]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

def distance_meters(lat_start, long_start, lat_finish, long_finish):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    start = to_mercator(lat_start, long_start)
    finish = to_mercator(lat_finish, long_finish)
    return start.distance(finish)

In [21]:
distance_meters(40.582363, 3.705431, 40.477264, 3.743003)

#(lat, lon) (lat, lon)

0    12427.681274
dtype: float64

In [22]:
df_bicimad.iloc[1, 2:3]

latitude     40.41731271011562
Name: 1, dtype: object

In [41]:
prueba1 = cleaning_df_garden.loc[:2]
prueba1 = pd.DataFrame(prueba1)
#prueba1 = prueba1.transpose()
prueba1

,title,@type,locality,postal-code,street-address,latitude,longitude
0,Jardines Gregorio Ordóñez,ParquesJardines,MADRID,28006,CALLE PRINCIPE DE VERGARA 68,40.433462,-3.678595
1,Jardines San Francisco el Grande,ParquesJardines/ColeccionesBotanicas,MADRID,28005,AVENIDA GRAN VIA DE SAN FRANCISCO 29,40.407143,-3.712167
2,Jardines de Andrés Saborit,ParquesJardines,MADRID,28003,CALLE RAIMUNDO FERNANDEZ VILLAVERDE 32,40.446133,-3.698970


In [24]:
(cleaning_df_garden.iloc[1, 5:6], cleaning_df_garden.iloc[1, 6:], df_bicimad.iloc[2, 2:3], df_bicimad.iloc[2, 3:4])

(latitude    40.407143
 Name: 1, dtype: object,
 longitude   -3.712167
 Name: 1, dtype: object,
 latitude     40.4205886
 Name: 2, dtype: object,
 longitude    -3.7058415
 Name: 2, dtype: object)

In [25]:
(float(cleaning_df_garden.loc[1, 'latitude']), cleaning_df_garden.loc[1, 'longitude'], float(df_bicimad.loc[1, 'latitude']), float(df_bicimad.loc[1, 'longitude']))

(40.40714274084601, -3.7121666160537057, 40.41731271011562, -3.701602938060457)

In [26]:
df = pd.DataFrame({'c1': [10, 11, 12], 'c2': [100, 110, 120]})
df = df.reset_index()  # make sure indexes pair with number of rows

for index, row in df.iterrows():
    print(row['c1'], row['c2'])

10 100
11 110
12 120


In [50]:
%%time
distance=[]
for index, row in cleaning_df_garden.iterrows():
    for index, j in df_bicimad.iterrows():
        distance.append(distance_meters(float(row['latitude']), float(row['longitude']), float(j['latitude']), float(j['longitude'])))

CPU times: total: 10min 38s
Wall time: 11min 57s


In [109]:
%%time
distance_int=[]
for index, row in cleaning_df_garden.iterrows():
    for index, j in df_bicimad.iterrows():
        distance_int.append(distance_meters(row['latitude'], row['longitude'], j['latitude'], j['longitude']))

CPU times: total: 10min 15s
Wall time: 12min 12s


In [110]:
distance_int

[0    3160.9304
 dtype: float64,
 0    3133.481106
 dtype: float64,
 0    3360.222755
 dtype: float64,
 0    3178.932155
 dtype: float64,
 0    2731.56093
 dtype: float64,
 0    2673.580654
 dtype: float64,
 0    2445.151861
 dtype: float64,
 0    2328.844212
 dtype: float64,
 0    1979.532898
 dtype: float64,
 0    3980.25144
 dtype: float64,
 0    1761.320808
 dtype: float64,
 0    2881.095899
 dtype: float64,
 0    3129.196068
 dtype: float64,
 0    3617.548456
 dtype: float64,
 0    3976.768435
 dtype: float64,
 0    3317.170695
 dtype: float64,
 0    3426.17579
 dtype: float64,
 0    3035.309858
 dtype: float64,
 0    2740.455456
 dtype: float64,
 0    2285.327086
 dtype: float64,
 0    2459.305458
 dtype: float64,
 0    3454.733576
 dtype: float64,
 0    3605.917204
 dtype: float64,
 0    3977.090581
 dtype: float64,
 0    3521.92944
 dtype: float64,
 0    3585.309723
 dtype: float64,
 0    2069.15937
 dtype: float64,
 0    2941.613249
 dtype: float64,
 0    2793.459004
 dtype: f

In [51]:
%%time
distance_prueba=[]
for index, row in prueba1.iterrows():
    for index, j in df_bicimad.iterrows():
        distance_prueba.append(distance_meters(row['latitude'], row['longitude'], j['latitude'], j['longitude']))

CPU times: total: 10.6 s
Wall time: 15.1 s


In [52]:
len(distance_prueba)

792

# Run from here

In [106]:
# Distancias resultantes en lista
distance

[0    3160.9304
 dtype: float64,
 0    3133.481106
 dtype: float64,
 0    3360.222755
 dtype: float64,
 0    3178.932155
 dtype: float64,
 0    2731.56093
 dtype: float64,
 0    2673.580654
 dtype: float64,
 0    2445.151861
 dtype: float64,
 0    2328.844212
 dtype: float64,
 0    1979.532898
 dtype: float64,
 0    3980.25144
 dtype: float64,
 0    1761.320808
 dtype: float64,
 0    2881.095899
 dtype: float64,
 0    3129.196068
 dtype: float64,
 0    3617.548456
 dtype: float64,
 0    3976.768435
 dtype: float64,
 0    3317.170695
 dtype: float64,
 0    3426.17579
 dtype: float64,
 0    3035.309858
 dtype: float64,
 0    2740.455456
 dtype: float64,
 0    2285.327086
 dtype: float64,
 0    2459.305458
 dtype: float64,
 0    3454.733576
 dtype: float64,
 0    3605.917204
 dtype: float64,
 0    3977.090581
 dtype: float64,
 0    3521.92944
 dtype: float64,
 0    3585.309723
 dtype: float64,
 0    2069.15937
 dtype: float64,
 0    2941.613249
 dtype: float64,
 0    2793.459004
 dtype: f

In [49]:
#df_distance = pd.DataFrame(distance)

In [ ]:
'''
crear columnas cada 203 rows
una lista = una columna
una columna cuando count llege a 203
if count == 203
    return new_column

'''

In [58]:
%%time
list_df=[]
count=0
list_list = []
for i in distance:
    if count != 264:
        count = count + 1
        list_list.append(i)
    if count == 264:
        count = 0
        list_df.append(list_list)
        list_list = []

len(list_df)

CPU times: total: 15.6 ms
Wall time: 36.7 ms


203

In [102]:
%%time
list_df

CPU times: total: 0 ns
Wall time: 0 ns


[[0    3160.9304
  dtype: float64,
  0    3133.481106
  dtype: float64,
  0    3360.222755
  dtype: float64,
  0    3178.932155
  dtype: float64,
  0    2731.56093
  dtype: float64,
  0    2673.580654
  dtype: float64,
  0    2445.151861
  dtype: float64,
  0    2328.844212
  dtype: float64,
  0    1979.532898
  dtype: float64,
  0    3980.25144
  dtype: float64,
  0    1761.320808
  dtype: float64,
  0    2881.095899
  dtype: float64,
  0    3129.196068
  dtype: float64,
  0    3617.548456
  dtype: float64,
  0    3976.768435
  dtype: float64,
  0    3317.170695
  dtype: float64,
  0    3426.17579
  dtype: float64,
  0    3035.309858
  dtype: float64,
  0    2740.455456
  dtype: float64,
  0    2285.327086
  dtype: float64,
  0    2459.305458
  dtype: float64,
  0    3454.733576
  dtype: float64,
  0    3605.917204
  dtype: float64,
  0    3977.090581
  dtype: float64,
  0    3521.92944
  dtype: float64,
  0    3585.309723
  dtype: float64,
  0    2069.15937
  dtype: float64,
  0    2

In [105]:
%%time
loc = pd.DataFrame(list_df)
loc

CPU times: total: 31.2 ms
Wall time: 31 ms


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263
0,0 3160.9304 dtype: float64,0 3133.481106 dtype: float64,0 3360.222755 dtype: float64,0 3178.932155 dtype: float64,0 2731.56093 dtype: float64,0 2673.580654 dtype: float64,0 2445.151861 dtype: float64,0 2328.844212 dtype: float64,0 1979.532898 dtype: float64,0 3980.25144 dtype: float64,0 1761.320808 dtype: float64,0 2881.095899 dtype: float64,0 3129.196068 dtype: float64,0 3617.548456 dtype: float64,0 3976.768435 dtype: float64,0 3317.170695 dtype: float64,0 3426.17579 dtype: float64,0 3035.309858 dtype: float64,0 2740.455456 dtype: float64,0 2285.327086 dtype: float64,0 2459.305458 dtype: float64,0 3454.733576 dtype: float64,0 3605.917204 dtype: float64,0 3977.090581 dtype: float64,0 3521.92944 dtype: float64,0 3585.309723 dtype: float64,0 2069.15937 dtype: float64,0 2941.613249 dtype: float64,0 2793.459004 dtype: float64,0 2705.863501 dtype: float64,0 3999.171116 dtype: float64,0 3686.424905 dtype: float64,0 3692.555079 dtype: float64,0 3320.459687 dtype: float64,0 3493.562393 dtype: float64,0 4139.032535 dtype: float64,0 4145.449248 dtype: float64,0 4696.996094 dtype: float64,0 4413.921038 dtype: float64,0 4180.701954 dtype: float64,0 3658.720801 dtype: float64,0 3292.227138 dtype: float64,0 3348.669469 dtype: float64,0 3720.575878 dtype: float64,0 3877.333732 dtype: float64,0 4664.951064 dtype: float64,0 4466.125031 dtype: float64,0 4182.92143 dtype: float64,0 4071.896129 dtype: float64,0 3821.441585 dtype: float64,0 3242.001024 dtype: float64,0 3329.469878 dtype: float64,0 3252.861013 dtype: float64,0 3732.02189 dtype: float64,0 2549.137517 dtype: float64,0 2824.977665 dtype: float64,0 3219.099049 dtype: float64,0 3821.281705 dtype: float64,0 2486.476797 dtype: float64,0 3004.054793 dtype: float64,0 1877.469 dtype: float64,0 1395.16627 dtype: float64,0 1513.693814 dtype: float64,0 1931.201281 dtype: float64,0 2224.04473 dtype: float64,0 3000.125962 dtype: float64,0 2692.385836 dtype: float64,0 2869.627135 dtype: float64,0 2022.570625 dtype: float64,0 2018.200908 dtype: float64,0 2234.621716 dtype: float64,0 2678.266901 dtype: float64,0 2424.959364 dtype: float64,0 2803.204722 dtype: float64,0 2956.51064 dtype: float64,0 3152.788566 dtype: float64,0 3667.113082 dtype: float64,0 3158.539462 dtype: float64,0 3160.934075 dtype: float64,0 3164.23876 dtype: float64,0 3076.967534 dtype: float64,0 2953.288177 dtype: float64,0 2289.770909 dtype: float64,0 3428.969439 dtype: float64,0 1340.768417 dtype: float64,0 2329.578093 dtype: float64,0 822.571494 dtype: float64,0 629.174158 dtype: float64,0 787.509798 dtype: float64,0 1102.362487 dtype: float64,0 815.902964 dtype: float64,0 1192.982471 dtype: float64,0 1479.931687 dtype: float64,0 1772.336793 dtype: float64,0 1528.654232 dtype: float64,0 1330.294988 dtype: float64,0 1261.261909 dtype: float64,0 1103.186329 dtype: float64,0 966.368523 dtype: float64,0 1102.471917 dtype: float64,0 1557.75852 dtype: float64,0 1495.241701 dtype: float64,0 949.660827 dtype: float64,0 1433.996655 dtype: float64,0 1222.552042 dtype: float64,0 1379.503506 dtype: float64,

In [99]:
distance_garden = []
for i in loc.iterrows():
    for j in i:
        distance_garden.append(j)
distance_garden.strip()

AttributeError: 'list' object has no attribute 'strip'

In [ ]:
distance=[]
for index, row in cleaning_df_garden.iterrows():
    for index, j in df_bicimad.iterrows():
        distance.append(distance_meters(float(row['latitude']), float(row['longitude']), float(j['latitude']), float(j['longitude'])))

In [ ]:
# problema: identificar antes de hacer el min la station

In [ ]:
# min of column(garden)= min and row(station) where it is

In [ ]:
# row = row + cell

In [ ]:
# row = name station index + cell

In [ ]:
# Transpose dataframe

In [ ]:
# select the min of each column or row

In [56]:
for i in distance:
    for j in range(203):
        count = 0
        count = count + 1
        if count == 203
        column= count +1
        

TypeError: object of type 'int' has no len()

In [38]:
df_distance.apply(args=[264])

NameError: name 'add_data' is not defined

In [ ]:
for h in range(height):
    for w in range(width):
        if maskImg[h][w] > 0:
            maskImg[h][w] = srcImg[h][w]

In [ ]:
lambda_function = lambda height, width, src, mask: [[src[h][w] if mask[h][w] > 0 else mask[h][w] for w in range(width)] for h in range(height)]


In [47]:
%%time
distance = [lambda x : (distance_meters(float(row['latitude']), float(row['longitude']), float(j['latitude']), float(j['longitude']))), (for index, row in cleaning_df_garden.iterrows() for index, j in df_bicimad.iterrows())]

SyntaxError: invalid syntax (<unknown>, line 1)

In [ ]:
distance = map(lambda x : for index, row in cleaning_df_garden.iterrows() for index, j in df_bicimad.iterrows() distance_meters(float(row['latitude']), float(row['longitude']), float(j['latitude']), float(j['longitude'])))

In [ ]:
for i in cleaning_df_garden:
    index = i
    for j in df_bicimad:
        index = j
        distance_meters(float(cleaning_df_garden.loc[index = i, 'latitude']), float(cleaning_df_garden.loc[i, 'longitude']), float(df_bicimad.loc[index = j, 'latitude']), float(df_bicimad.loc[j, 'longitude']))
        

In [ ]:
cleaning_df_garden['latitude']

In [ ]:
for i in cleaning_df_garden['coordinates']:
    for j in df_bicimad['coordinates']:
        distance_meters(i[0], i[1], j[0], j[1])
        
    